***

## Problem Set 8 Solutions

>CSC427, semester 202 (jan-may 2020)
<br>
burton rosenberg
<br>
univ of miami
<br>
(c) 2020 all rights reserved

<br>
_*Created: 20 April 2020*_
<br>
_*Last update: 1 May 2020*_

***



## Student's Name: (your name here)



### Statement of the project

In this final project, the study of P and NP will be practiced.

Your tasks are to,

1- Write a polynomial time predicate VerifyPath.find_if_path, that given a graph and two vertices,
   returns true if the two vertices are vertices in the graph, and there is a path in the graph connecting
   the two vertices. Else it returns False.

2- Write a polynomial time predicate VerifyKCover.verify, that given a graph and a list of vertices, 
   returns true if the list of vertices is a vertex-covering of the graph. That is, ever edge of the
   graph has at least one end vertice in the given cover. Else it returns False.
   
3- Write an exponential time search algoirthm, SolveKCover.search_brute_force, that given a graph and
   an integer k, returns a vertex-covering with exactly k vertices of the graph, or else returns the
   empty list. The algorithm will work by enumerating all subsets of k vertices from the set of all
   the graph's vertices, and will apply the verifying on each such candidate solution.
   
4- Write a polynomial time reduction Reduction.reduce from 3-SAT to Vertex Cover, that is, which given
   an instance of 3-SAT produces a graph and a number k, such that 3-SAT is satisfiable if and only
   if the graph has a k covering.
   
5- Write a polynomail time function Reduction.pullback, which given a 3-SAT to Vertex Cover reduction,
   and a k cover, produces a satisfying assignment for the 3-SAT instance, the subject of the reduction.
   
6- Put it all together: Write a 3-SAT solver that solves a 3-SAT instance by reducing it to a Vertex Cover
   instance, solves the instance by brute force, and pulls back the vertex cover solution to a satisfying 
   assignment for the 3-SAT instance.
   

   
   

## Description of data structure

### Description of a graph

1- A graph is made up of vertices and edges. Each edge is a pair of vertices

2- A vertex will be represented as a string, to be thought of as the name of the vertex.

3- An edge will be represented as a Python pair. The two members of the pair will be the vertex name.

4- A graph will be a Python list of pairs.

5- The data structure can represent a directed graph or of an undirected graph. The algorithm makes 
   the distinction.

   - The PATH algorithm is for directed graphs, hence it interprets the data structure where the
     edges are directed and go from the first element of the pair to the second. 
     
   - The k-VC algorithm is for undirected graphs, hence it interprets the data structure
     where the edges are not directed.


In [1]:
# Example of a graph. The textbook graph of Figure 0.16 is represented -

figure_0_16 = [
    ('1','2'),
    ('1','5'),
    ('2','1'),
    ('2','4'),
    ('5','4'),
    ('5','6'),
    ('6','1'),
    ('6','3')
]

# here are some examples where path would be true, for this graph

figure_0_16_true = [
    ('6','3'),
    ('6','4'),
    ('2','5'),
    ('1','3')
]

# and here are some where path would be false

figure_0_16_false =[
    ('3','6'),
    ('4','1')
]



### Description of a 3-SAT instance


1- A 3-SAT instance is made up of clauses, signed variables, and variables.

2- A variable will be represented as a string, to be thought of as the name of the variable.

3- A signed variable is a Python pair. The first element is a variable name and the second
   element is 0 or 1. 
 * If 0, the sign of the variable is positive, the variable is appearing
   without negation; 
 * If 1, the sign of the variable is negative, the variable is appearing
   negated.

4- A clause is Python list of 3 signed variables.

5- An instance of 3-SAT is a Python list of clauses.


In [2]:
# The 3-SAT in Figure 7.33 is represented - 

figure_7_45 = [
    [('x1',0),('x1',0),('x2',0)],
    [('x1',1),('x2',1),('x2',1)],
    [('x1',1),('x2',0),('x2',0)]
]

### Description of a Truth Assignment

A _truth assignment_ will be represented as a dictionary with variable names, as strings,
as keys, and values either True or False.

The truth assignment data structure is used when reporting a satisfying assignment for a formula.
That a formula is not satisfiable *can* be indicated by the empty dictionary, {}.


In [3]:
# examples

truth_assignment_1 = {'x1':True, 'x2':False, 'x3':True}
truth_assignment_2 = {}



### Description of a k-VC instance
 
An k vertex cover instance is a Python pair.

   * The first element of the pair is the integer k
   * The second element of the pair is a graph



### Description of a vertex cover

A vertex cover is a list of vertices. 

If a k-vertex cover does not exist for a graph, the vertex cover *can* indicate that by being the
empty list, [].



In [4]:
# examples:

k_vc_1 = (2,[('a','b'),('b','c'),('c','a')])   # this k vertex cover instance has a solution
k_vc_2 = (1,[('a','b'),('b','c'),('c','a')])   # this k vertex cover instance does not have a solution


cover_1 = ['a','b'] # a vertex cover that is a solution to k_vc_1
cover_2 = []  # the empty vertex cover, indicating k_vc_2 has no solution.


***

## Student workspace



### P-Time Path algorithm

In [5]:

class VerifyPath:
    
    @staticmethod
    def find_if_path(graph,source,dest):

        def add_to_dict(d,e,is_directed=False):
            (e1,e2) = e
            if e1 not in d:
                d[e1] = []
            d[e1].append(e2)
            if is_directed:
                if e2 not in d:
                    d[e2] = []
                d[e2].append(e1)

        edge_dict = {}
        for edge in graph:
            add_to_dict(edge_dict,edge)

        if source not in edge_dict:
            return False

        visited = set([source])
        q = edge_dict[source]

        while len(q)>0:
            v = q.pop()
            if v not in visited:
                visited.add(v)
                if v in edge_dict:
                    q += edge_dict[v]

        return dest in visited


### P-Time Vertex Cover verifier

In [6]:
class VerifyKCover:
    
    def __init__(self,graph,verbose=set()):
        self.graph = graph
        self.verbose = verbose

    def verify(self,cover):
        """
        input:
            a vertex cover
            and the instance variable self.graph
        returns:
            True/False
        
        method:
            verify that the vertex list given is a k-cover
        """
        c = set(cover)
        for edge in self.graph:
            if edge[0] not in c and edge[1] not in c:
                if 'verify' in self.verbose: 
                    print('verify: edge {} not covered'.format(edge))
                return False
            
        return True



### Exp-Time Vertex Cover solver

In [7]:

class SolveKCover:
    
    def __init__(self,graph,verbose=set()):
        self.graph = graph
        self.verbose = verbose
        self.verifier = VerifyKCover(graph,verbose)
        
        self.iterate_idx = 0
        self.k = 0
        
        # make a list of vertices of the graph
        v = set()
        for edge in graph:
            v.add(edge[0])
            v.add(edge[1])
        self.vertices = list(v)
       
    def get_vertices(self):
        return self.vertices
    
    def get_verifier(self):
        return self.verifier
    
    def get_ith_bit(self,i,n):
        return (n//2**i)%2

    def count_ones(self,n):
        count = 0
        while n>0:
            count += n%2
            n = n//2
        return count

    def new_iteration(self,k):
        self.k = k
        self.iterate_idx = 0
        
    def next_iterate(self):
        idx = self.iterate_idx + 1
        while (self.count_ones(idx)!=self.k):
            idx += 1
        self.iterate_idx = idx
        return idx

    def select_vertices_by_index(self,idx):
        l = len(self.vertices)
        i = 0
        selected = []
        while idx>0 and i<l:
            if idx%2 == 1:
                selected.append(self.vertices[i])   
            idx = idx//2
            i +=1
        return selected

    def search_brute_force(self,k):
        """
        input:
            an integer k, 
            and the instance variable self.graph
        returns:
            a vertex cover
        """

        n_vertices = len(self.vertices)
        if k>n_vertices:
            return []
        if k<=0:
            return []

        assert(k>0)
        self.new_iteration(k)
        idx = self.next_iterate()
        largest_idx = 2**n_vertices

        while idx <= largest_idx:
            selected = self.select_vertices_by_index(idx)
            if 'search_brute_force' in self.verbose:
                print('search_brute_force: candidate {}'.format(selected))
            res = self.verifier.verify(selected)
            if res:
                return selected
            idx = self.next_iterate()
        return []



### Some examples of how to use the enumerating functions

In [8]:
# using the functions implemented in SOlveKCover to
# enumerate the integers with binary representation 
# containing exact one 1 bit. e.g. 1, 2, 4, ...
                  
def iterator_pure_powers(n_max):
    skc = SolveKCover([('a','a')])
    skc.new_iteration(1)
    idx = skc.next_iterate()
    while idx<n_max:
        print(idx)
        idx = skc.next_iterate()

iterator_pure_powers(5000)

# using the functions implemented in SOlveKCover to
# enumerate all subsets of 3 items from a set of n items
                      
def enumerate_3_sets(n):

    graph = []
    for i in range(n):
        var = 'var_{}'.format(i)
        graph.append((var,var))

    skc = SolveKCover(graph)
    skc.new_iteration(3)
    idx = skc.next_iterate()
    idx_max = 2**len(graph)
    print('number of s/sets = {}'.format(int(n*(n-1)*(n-2)/6)))
    i = 1
    while idx<idx_max:
        print('{}:\t{}'.format(i,skc.select_vertices_by_index(idx)))
        idx = skc.next_iterate()
        i += 1

                      
enumerate_3_sets(6)


1
2
4
8
16
32
64
128
256
512
1024
2048
4096
number of s/sets = 20
1:	['var_4', 'var_2', 'var_1']
2:	['var_4', 'var_2', 'var_0']
3:	['var_4', 'var_1', 'var_0']
4:	['var_2', 'var_1', 'var_0']
5:	['var_4', 'var_2', 'var_3']
6:	['var_4', 'var_1', 'var_3']
7:	['var_2', 'var_1', 'var_3']
8:	['var_4', 'var_0', 'var_3']
9:	['var_2', 'var_0', 'var_3']
10:	['var_1', 'var_0', 'var_3']
11:	['var_4', 'var_2', 'var_5']
12:	['var_4', 'var_1', 'var_5']
13:	['var_2', 'var_1', 'var_5']
14:	['var_4', 'var_0', 'var_5']
15:	['var_2', 'var_0', 'var_5']
16:	['var_1', 'var_0', 'var_5']
17:	['var_4', 'var_3', 'var_5']
18:	['var_2', 'var_3', 'var_5']
19:	['var_1', 'var_3', 'var_5']
20:	['var_0', 'var_3', 'var_5']


### Reducing 3-SAT to k-VC

In [9]:
import re

class Reduction:

    def __init_(self,verbose=set()):
        self.verbose = verbose

    @staticmethod
    def reduce(instance_3sat):
        """
        input:
            A 3-SAT instance
        returns:
            A k-VC instance
            
        Method:
            The widget: create C_3 for every clause, and K_2 for each variable.
            Assocate each of the vertices in the C_3 with a variable in the clause, and the
            pair of vertices in the K_2 with the postive and negated forms of the variable.
        
            Also create an edge from the vertex in C_3 for variable v (resp not v) to the
            vertices in K_2 for variable v (resp not v)
        
            Any cover must be at least n+2m, for n variables and m clauses. 
            The cover sought has k = n+2m.
        """
        
        # collect up the nodes
        all_var = set()
        for clause in instance_3sat:
            for var in clause:
                all_var.add(var[0])
                
        instance_vc = []
        # create the K_2's
        for var_name in all_var:
            instance_vc.append(('{}_T'.format(var_name),'{}_F'.format(var_name)))
        # create the C_3's and connections to K_2's
        clause_num = 1
        var_name = ['' for i in range(3)]
        for clause in instance_3sat:
            i = 0
            for var in clause:
                if var[1]==1:
                    var_code = '{}_F'.format(var[0])
                    var_name[i] = '{}_{}_{}'.format(var_code,clause_num,i)
                    instance_vc.append((var_name[i],var_code))
                else:
                    var_code = '{}_T'.format(var[0])
                    var_name[i] = '{}_{}_{}'.format(var_code,clause_num,i)
                    instance_vc.append((var_name[i],var_code))
                i += 1
            instance_vc += [(var_name[0],var_name[1]),(var_name[1],var_name[2]),(var_name[2],var_name[0])]
            clause_num += 1
                
        return (len(all_var)+2*(clause_num-1),instance_vc)
    
    @staticmethod
    def pullback(vertex_cover):
        """
        input:
            A vertex cover for some graph.
            Note: it is possible that the graph itself is irrelevant
        return:
            A truth assignment.
        """
        assignment = {}
        p_t = re.compile('(.*)_T$')
        p_f = re.compile('(.*)_F$')
        for vertex in vertex_cover:
            r = p_t.match(vertex)
            if r:
                 assignment[r.group(1)] = True
            r = p_f.match(vertex)
            if r:
                 assignment[r.group(1)] = False
        return assignment
    
    @staticmethod
    def solve(instance_3sat):
        """
        input:
            A 3-sat instance. 
        returns: 
            A truth assignment.
        """

        k, vc_graph = Reduction.reduce(instance_3sat)
        skc = SolveKCover(vc_graph)
        solution = skc.search_brute_force(k)
        if solution:
            solution = Reduction.pullback(solution)

        return solution
    


### Testing region

In [10]:


def test_get_ith_bit(m):
    for n in range(m):
        sum_up = 0
        power_of = 1
        for i in range(m):
            bit = skc.get_ith_bit(i,n)
            sum_up += power_of * bit
            power_of *= 2
        if sum_up != n:
            print('bad')
            return False
    print('good')
    return True

def test_count_ones(m):
    for n in range(m):
        print("{}: {}".format(n,skc.count_ones(n)))


def check_next_with_k_ones(k_max,n_max):
    for k in range(1,k_max):
        print("checking k = {}".format(k))
        k_ones = 0
        while k_ones<n_max:
            k_ones = skc.next_with_k_ones(k,k_ones)
            if skc.count_ones(k_ones) != k :
                print("error on number {}".format(k_ones))
                return False
    print("check_next_with_k_ones: done")
    return True

def test_solvekcover_enumerate_functions():       
    test_get_ith_bit(100)
    test_count_ones(8)
    iterator_pure_powers(2000)
    #check_next_with_k_ones(20,2000)

#test_solvekcover_enumerate_functions()

graph = [('a','b'),('b','c'),('c','a'),('d','e')]
print(VerifyPath.find_if_path(graph, 'a', 'b'))
print(VerifyPath.find_if_path(graph, 'a', 'e'))    

g1 = [('a','b'),('b','c'),('c','d'),('d','a'),('d','e'),('e','f'),('f','g1'),('f','g2')]
v = VerifyKCover(g1,verbose = set(['verify']))
print(v.verify(['b','d','f']))
print(v.verify(['g1','g2','e','a','c']))

s = SolveKCover(g1, verbose=set(['search_brute_force']) )
solution = s.search_brute_force(5)
if len(solution)>0:
    print(s.get_verifier().verify(solution))

graph = [('a','b'),('b','c'),('c','a'),('d','e')]
skc = SolveKCover(graph)



True
False
True
True
search_brute_force: candidate ['b', 'd', 'c', 'e', 'g1']
search_brute_force: candidate ['b', 'd', 'c', 'e', 'f']
True


In [11]:
figure_7_45 = [
    [('x1',0),('x1',0),('x2',0)],
    [('x1',1),('x2',1),('x2',1)],
    [('x1',1),('x2',0),('x2',0)]
]
Reduction.reduce(figure_7_45)
print(Reduction.solve(figure_7_45))

sat_1 = [
    [('a',0),('b',0),('c',0)]
]
print(Reduction.solve(sat_1))

sat_2= [
    [('a',1),('a',1),('a',1)]
]
print(Reduction.solve(sat_2))

sat_3 = [
    [('a',0),('b',0),('c',0)],
    [('a',1),('b',1),('c',1)]
]
print(Reduction.solve(sat_3))

{'x1': False, 'x2': True}
{'a': False, 'b': True, 'c': True}
{'a': False}
{'a': False, 'b': True, 'c': True}


***

## A Basic Test

In [12]:
def verify_path_tests(graph,tests):
    print("\n*** begin verify_path_tests ***")
    for test in tests:
        if VerifyPath.find_if_path(graph, test[1], test[2]) != test[0]:
            print('\tBAD: ({},{}) gives {}'.format(test[1],test[2],test[0]))
            print("*** fails verify_path_tests ***\n")
            return False
        print('\tOK: ({},{}) gives {}'.format(test[1],test[2],test[0]))
    print("*** passes verify_path_tests ***\n")
    return True


def verity_k_cover_tests(graph,tests):
    print("\n*** begin verify k cover tests ***")
    v = VerifyKCover(graph)
    for test in tests:
        if v.verify(test[1]) != test[0]:
            print('\tBAD: {} gives {}'.format(test[1],test[0]))
            print("*** fails verify k cover tests ***\n")
            return False
        print('\tOK: {} gives {}'.format(test[1],test[0]))
    print("*** passes verify k cover tests ***\n")
    return True


def solve_k_cover_example(graph,ks):
    print("\n*** begin SolveKCover tests ***")
    s = SolveKCover(graph)
    for k in ks:
        solution = s.search_brute_force(k[1])
        if k[0]:
            if (len(solution)>0) and not s.get_verifier().verify(solution):
                print('\tBAD: {}'.format(k[1]))
                print("*** fails SolveKCover tests ***\n")
                return False
            if len(solution)==0:
                print('\tBAD: {}'.format(k[1]))
                print("*** fails SolveKCover tests ***\n")
                return False                
        if not k[0] and (len(solution)>0):
            print('\tBAD: {}'.format(k[1]))
            print("*** fails SolveKCover tests ***\n")
            return False
        print('\tOK: {}'.format(k[1]))
    print("*** passes SolveKCover tests ***\n")
    return True




# Example of a graph. The textbook graph of Figure 0.16 is represented -

figure_0_16 = [
    ('1','2'),
    ('1','5'),
    ('2','1'),
    ('2','4'),
    ('5','4'),
    ('5','6'),
    ('6','1'),
    ('6','3')
]

# here are some examples where path would be true, for this graph

figure_0_16_answers = [
    (True,'6','3'),
    (True,'6','4'),
    (True,'2','5'),
    (True,'1','3'),
    (False,'3','6'),
    (False,'4','1')
]

gr1 = [('a','b'),('b','c'),('c','a'),('d','e')]
gr1_answers = [(False,'a','d'), (False,'e','d'),
              (True,'c','b'),(True,'a','c')]

gr2 = [('a','b1'),('b1','c1'),('a','b2'),('b2','c2'),('b3','a'),('c3','b3'),('e','e')]
gr2_answers = [(True,'e','e'),(True,'a','c1'),(True,'c3','c1'),(False,'c3','e'),(False,'c2','a')]


gr3 = [('a','b'),('b','c'),('c','d'),('d','a'),('d','e'),('e','f'),('f','g1'),('f','g2')]
gr3_answers = [[True,['b','d','f']],[True,['g1','g2','e','a','c']],[False,['c','e','g2']]]
gr3_k_solvable = [[False,1],[False,2],[True,3],[True,4],[True,5]]

verify_path_tests(figure_0_16,figure_0_16_answers)
verify_path_tests(gr1,gr1_answers)
verify_path_tests(gr2,gr2_answers)
verity_k_cover_tests(gr3,gr3_answers)

solve_k_cover_example(gr3,gr3_k_solvable)


*** begin verify_path_tests ***
	OK: (6,3) gives True
	OK: (6,4) gives True
	OK: (2,5) gives True
	OK: (1,3) gives True
	OK: (3,6) gives False
	OK: (4,1) gives False
*** passes verify_path_tests ***


*** begin verify_path_tests ***
	OK: (a,d) gives False
	OK: (e,d) gives False
	OK: (c,b) gives True
	OK: (a,c) gives True
*** passes verify_path_tests ***


*** begin verify_path_tests ***
	OK: (e,e) gives True
	OK: (a,c1) gives True
	OK: (c3,c1) gives True
	OK: (c3,e) gives False
	OK: (c2,a) gives False
*** passes verify_path_tests ***


*** begin verify k cover tests ***
	OK: ['b', 'd', 'f'] gives True
	OK: ['g1', 'g2', 'e', 'a', 'c'] gives True
	OK: ['c', 'e', 'g2'] gives False
*** passes verify k cover tests ***


*** begin SolveKCover tests ***
	OK: 1
	OK: 2
	OK: 3
	OK: 4
	OK: 5
*** passes SolveKCover tests ***



True